In [ ]:
!unzip hse-itmo-acoustic-event-detection-2024.zip

In [ ]:
# path to your train/test/meta folders
DATA_PATH = '.'

# names of valuable files/folders
train_meta_fname = 'train.csv'
test_meta_fname = 'sample_submission.csv'
train_data_folder = './audio_train/train'
test_data_folder = './audio_test/test'

In [ ]:
import numpy as np
import os
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import torchaudio
import torchvision
from torchaudio import transforms
#from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from tqdm import tqdm

In [ ]:
# set seeds
import random
import numpy as np

random.seed(42)
np.random.seed(42)
torch.manual_seed(42)
torch.cuda.manual_seed(42)
torch.backends.cudnn.deterministic = True

In [ ]:
df_train = pd.read_csv(os.path.join(DATA_PATH, train_meta_fname))
df_test = pd.read_csv(os.path.join(DATA_PATH, test_meta_fname))
df_train.head(2)

,fname,label
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping
1,00d77b917e241afa06f1.wav,Squeak


In [ ]:
n_classes = df_train.label.nunique()
print(n_classes)
classes_dict = {cl:i for i,cl in enumerate(df_train.label.unique())}

label2id = classes_dict
id2label = {idx: label for label, idx in label2id.items()}

df_train['label_encoded'] = df_train.label.map(classes_dict)
df_train.head()

41


,fname,label,label_encoded
0,8bcbcc394ba64fe85ed4.wav,Finger_snapping,0
1,00d77b917e241afa06f1.wav,Squeak,1
2,17bb93b73b8e79234cb3.wav,Electric_piano,2
3,7d5c7a40a936136da55e.wav,Harmonica,3
4,17e0ee7565a33d6c2326.wav,Snare_drum,4


In [ ]:
from transformers import AutoFeatureExtractor, ASTForAudioClassification

feature_extractor = AutoFeatureExtractor.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/297 [00:00<?, ?B/s]

In [ ]:
# https://github.com/lukemelas/EfficientNet-PyTorch
class SanyaModel(nn.Module):

    def __init__(self, sample_rate=16000, n_classes=41):
        super().__init__()

        self.model = ASTForAudioClassification.from_pretrained("MIT/ast-finetuned-audioset-10-10-0.4593")

        # Define a linear layer for classification
        self.classifier = nn.Linear(self.model.config.hidden_size, n_classes)

    def forward(self, input_values):
        outputs = self.model(input_values.squeeze(1))

        return outputs

    def inference(self, x):
        x = self.forward(x.input_values)
        x = F.softmax(x)
        return x

In [ ]:
def prepare_input(audio_array, sampling_rate):
    # Process audio input with the processor
    inputs = feature_extractor(audio_array, sampling_rate=sampling_rate, return_tensors="pt", padding=True)
    return inputs

class EventDetectionDataset(Dataset):
    def __init__(self, data_path, x, y=None):
        self.x = x
        self.y = y
        self.data_path = data_path

    def __len__(self):
        return len(self.x)

    def __getitem__(self, idx):
        #print(idx)

        path2wav = os.path.join(self.data_path, self.x[idx])
        #print(path2wav)
        waveform, sample_rate = torchaudio.load(path2wav, normalize=True)
        waveform = prepare_input(waveform.squeeze().numpy(), sample_rate)
        if self.y is not None:
            return waveform, self.y[idx]
        return waveform


In [ ]:
X_train, X_val, y_train, y_val = train_test_split(df_train.fname.values, df_train.label_encoded.values,
                                                  test_size=0.2, random_state=42)
train_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_train, y_train),
                        batch_size=32
                )
val_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, train_data_folder), X_val, y_val),
                        batch_size=32
                )
test_loader = DataLoader(
                        EventDetectionDataset(os.path.join(DATA_PATH, test_data_folder), df_test.fname.values, None),
                        batch_size=1, shuffle=False
                )

In [ ]:
def eval_model(model, eval_dataset):
    model.eval()
    forecast, true_labs = [], []
    with torch.no_grad():
        for wavs, labs in tqdm(eval_dataset):
            wavs, labs = wavs.cuda(), labs.detach().numpy()
            true_labs.append(labs)
            outputs = model.inference(wavs)

            outputs = outputs.detach().cpu().numpy().argmax(axis=1)
            forecast.append(outputs)
    forecast = [x for sublist in forecast for x in sublist]
    true_labs = [x for sublist in true_labs for x in sublist]
    return f1_score(forecast, true_labs, average='macro')

In [ ]:
model = SanyaModel().cuda()

In [ ]:
model.model.config.label2id = label2id
model.model.config.id2label = id2label

In [ ]:
import gc

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

for epoch in range(3):
    for batch in tqdm(train_loader):
        input_values = batch[0]
        labels = batch[1]

        input_values['input_values'] = input_values['input_values'].cuda()
        labels = labels.cuda()

        logits = model(**input_values)

        # Compute loss and update weights
        loss = criterion(logits.logits, labels)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        gc.collect()

    print(f"Epoch {epoch+1}, Loss: {loss.item()}")

In [ ]:
forecast = []
with torch.no_grad():
    for wavs in tqdm(test_loader):
        inputs = wavs

        inputs['input_values'] = inputs['input_values'].cuda()

        # Perform inference
        with torch.no_grad():
            outputs = model(**inputs)

        predicted_class_idx = torch.argmax(outputs.logits).item()
        predicted_class_label = model.model.config.id2label[predicted_class_idx]

        forecast.append(predicted_class_label)

        print(f"Predicted class: {predicted_class_label}")

forecast = pd.Series(forecast)
df_test['label'] = forecast
df_test.to_csv('Sanya.csv', index=None)

  0%|          | 1/3790 [00:00<08:54,  7.09it/s]

././audio_test/test/973a8b381d1875ebe120.wav
Predicted class: Knock
././audio_test/test/052e1cbeb4391d6af234.wav


  0%|          | 3/3790 [00:00<09:18,  6.78it/s]

Predicted class: Acoustic_guitar
././audio_test/test/16eb98d96319450e0949.wav
Predicted class: Knock
././audio_test/test/ff13c136c04f274229a5.wav


  0%|          | 6/3790 [00:00<06:55,  9.10it/s]

...

Predicted class: Shatter
././audio_test/test/faab0a36d541f57eb304.wav
Predicted class: Trumpet
././audio_test/test/46d13b5e5e78951e056a.wav
Predicted class: Clarinet
././audio_test/test/959206e695c2ceed3d3b.wav


100%|█████████▉| 3788/3790 [06:11<00:00, 10.50it/s]

Predicted class: Bass_drum
././audio_test/test/3624e1c6d306945d7ba9.wav
Predicted class: Double_bass
././audio_test/test/4a22b7dd3f519e2a9111.wav
Predicted class: Snare_drum
././audio_test/test/a2c95e48acc6a3ec2696.wav


100%|██████████| 3790/3790 [06:11<00:00, 10.21it/s]

Predicted class: Double_bass
././audio_test/test/d660e01265224e7d27f0.wav
Predicted class: Oboe
